# Import Widget Libraries

In [1]:
%load_ext autoreload   
%autoreload 2
import magneton
from magneton.core.idom_loader import reload_bundle
from magneton.core.widget import WidgetModel

magneton_client: 0.0.0


# Import Graph Querying Library

In [2]:
from magneton import Service
service = Service(kh = 'onet_chi_replica:bc1628ba')

In [3]:
schema = service.get_kh_edge_list()

# Create an Overview/Exploration Widget

In [4]:
from magneton import Explorer


def init(): 
    # Fetch initial view data
    return {
        "subgraph": None,
        "children": service.get_children_node_distributions(),
        "relations": service.get_relation_distribution(),
    }

def focus(node, panel):
    # node can be in one of two formats:
    # node : { "node_label": str, "node_property": str, "node_property_value": str }
    # node : { "node_label": str, "title": str, "uuid": str }
    
    # Since the service functions use the first format, node must first
    # be converted to the proper format
    if "title" in node:
        node = {
            "node_label": node["node_label"],
            "node_property": "title",
            "node_property_value": node["title"]
        }
        
    # TODO: Return nodes in a consistent format from the service
    #       functions to streamline the above
    
    
    # Fetch the neighborhood around the input node 
    result = service.get_node_neighborhood(node)
    
    data = { "subgraph": None }
    if panel != "schema":
        data["schema"] = result["schema"]
    if panel != "children":
        # TODO: Accept a consistent input format (e.g., same as get_node_neighborhood)
        data["children"] = service.get_children_node_distributions(
            node
        )

    # Use a list comprehension to convert the return format of the relation_dist 
    # to a bar-chart format
    data["relations"] = [
        {"x": relation["label"], "y": relation["count"], "type": type, "relation": { "type": relation["label"], "direction": type } }
        for type, relations in result["relation_dist"].items()
        for relation in relations
    ]
    # TODO: Move conversion code into service
    
    return data


# Create widget 
explorer = Explorer({
        "init": init,
        "focus": focus
    }, schema)

In [5]:
reload_bundle()
explorer.show()

component(10c3e19a0, self=<magneton.widgets.StatefulWidgetBase.StatefulWidgetBase object at 0x10c3e0e80>)

LayoutWidget(Layout(component(10c3e19a0, self=<magneton.widgets.StatefulWidgetBase.StatefulWidgetBase object a…

In [8]:
reload_bundle()
prov = explorer.history()
prov.show()

component(10c4588e0, self=<magneton.widgets.StatefulWidgetBase.StatefulWidgetBase object at 0x10c3c6ac0>)

LayoutWidget(Layout(component(10c4588e0, self=<magneton.widgets.StatefulWidgetBase.StatefulWidgetBase object a…

In [ ]:
explorer.export_selection()

In [ ]:
print(service.get_node_parents({'node_label': 'occupation',
  'node_property': 'title',
  'node_property_value': 'Insurance Industry Occupations'}))

In [ ]:
explorer.debug()

In [ ]:
# def focus_relation(state, relation):
#     node = WidgetModel.unproxy(state.focus_node)
#     # node can be in one of two formats:
#     # node : { "node_label": str, "node_property": str, "node_property_value": str }
#     # node : { "node_label": str, "title": str, "uuid": str }
    
#     # Since the service functions use the first format, node must first
#     # be converted to the proper format
#     if node is not None and "title" in node:
#         node = {
#             "node_label": node["node_label"],
#             "node_property": "title",
#             "node_property_value": node["title"]
#         }
        
#     # TODO: Return nodes in a consistent format from the service
#     #       functions to streamline the above
    
#     # Fetch the neighborhood around the focus node/relation
#     result = service.get_relation_neighborhood(node, relation)
#     data = {}
#     data['schema'] = result["schema"]
#     data['children'] = [
#         {"x": x, "y": y} for x, y in result["node_dist"].items()
#     ]
#     return data

# View 0: Send nodes rather than labels in schema

In [ ]:
schema

# View 0: Click a node on minimap

In [ ]:
node ={'node_label': 'occupation',
       'node_property': 'title', 
       'node_property_value': 'occupation'}

service.get_children_node_distributions('occupation', 'title', 'Healthcare Occupations')

# View 0: Click a bar on node_dist

In [ ]:
node ={'node_label': 'occupation',
       'node_property': 'title', 
       'node_property_value': 'Allied Health Occupations'}

In [ ]:
nn = service.get_node_neighborhood(node)
nn

# View 0: Click a bar on relation dist

In [ ]:
node ={'node_label': 'occupation',
       'node_property': 'title', 
       'node_property_value': 'Fire Alarm Technicians'}

relation = {}
relation['type'] = 'has_core_skill'
relation['direction'] = 'out'

In [ ]:
service.get_relation_neighborhood(node, relation)

In [ ]:
service.get_relation_neighborhood(None, relation)

# View 1: Click a node on zoomed in graph

# View 1: Click a node on node_dist

# View 1: Click a node on relation dist